In [1]:
import gensim

In [2]:
import os, sys
import tqdm

def iterate_data_files(from_dtm, to_dtm):
    from_dtm, to_dtm = map(str, [from_dtm, to_dtm])
    read_root = os.path.join('./', 'read')
    for fname in os.listdir(read_root):
        if len(fname) != len('2018100100_2018100103'):
            continue
        if from_dtm != 'None' and from_dtm > fname:
            continue
        if to_dtm != 'None' and fname > to_dtm:
            continue
        path = os.path.join(read_root, fname)
        yield path, fname
 
data = [];
 
files = sorted([path for path, _ in iterate_data_files('2018100100', '2019022200')])
 
for path in tqdm.tqdm(files, mininterval=1):
    for line in open(path):
        tokens = line.strip().split()
        read_datetime = path[7:17]
        user_id = tokens[0]
        reads = tokens[1:]
        for item in reads:
            data.append([read_datetime, user_id, item])

100%|██████████| 3456/3456 [00:32<00:00, 107.94it/s]


In [3]:
import pandas as pd
 
read_df = pd.DataFrame(data)
read_df.columns = ['date', 'user_id', 'article_id']
 
read_df.head()

,date,user_id,article_id
0,2018100100,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91
1,2018100100,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31
2,2018100100,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49
3,2018100100,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95
4,2018100100,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140


In [128]:
metadata = pd.read_json('./meta/' + 'metadata.json', lines=True)
metadata.head()

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
0,8982,@bookdb,"사진으로 옮기기에도 아까운, 리치필드 국립공원","[여행, 호주, 국립공원]",https://brunch.co.kr/@bookdb/782,세상 어디에도 없는 호주 Top 10,1474944427000,782,@bookdb_782
1,12081,@kohwang56,[시] 서러운 봄,"[목련꽃, 아지랑이, 동행]",https://brunch.co.kr/@kohwang56/81,,1463092749000,81,@kohwang56_81
2,0,@hannahajink,무엇을 위해,[],https://brunch.co.kr/@hannahajink/4,무엇 때문에,1447997287000,4,@hannahajink_4
3,16315,@bryceandjuli,싫다,"[감정, 마음, 위로]",https://brunch.co.kr/@bryceandjuli/88,,1491055161000,88,@bryceandjuli_88
4,29363,@mijeongpark,Dubliner#7,"[유럽여행, 더블린, 아일랜드]",https://brunch.co.kr/@mijeongpark/34,#7. 내 친구의 집은 어디인가,1523292942000,34,@mijeongpark_34


In [129]:
userdata = pd.read_json('./meta/' + 'users.json', lines=True)
userdata.head()

,keyword_list,following_list,id
0,[],"[@perytail, @brunch]",#901985d8bc4c481805c4a4f911814c4a
1,[],"[@holidaymemories, @wadiz, @sciforus, @dailydu...",#1fd89e9dcfa64b45020d9eaca54e0eed
2,[],"[@commerceguy, @sunsutu, @kakao-it, @joohoonja...",#1d94baaea71a831e1f33e1c6bd126ed5
3,[],"[@amberjeon48, @forsy20, @nemotokki, @hawann, ...",#04641c01892b12dc018b1410e4928c0d
4,[],"[@dwcha7342, @iammento, @kakao-it, @dkam, @ant...",#65bcaff862aadff877e461f54187ab62


In [16]:
from konlpy.tag import *

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()

In [ ]:
%%time
metadata['nouns_list'] = metadata['title'].apply(lambda x: kkma.nouns(x))

In [63]:
metadata['doc_list'] = metadata['user_id'].apply(lambda x: [x]) + metadata['nouns_list'] + metadata['keyword_list']

In [81]:
metadata.to_csv('metadata.csv', index=False)

In [109]:
metadata = pd.read_csv('metadata.csv')

/home/ubuntu/anaconda3/envs/BrunchRec/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [110]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [291]:
metadata = metadata[metadata['id'].isin(read_df['article_id'].unique())].reset_index(drop=True)
metadata['doc_list'] = metadata['magazine_id'].apply(lambda x: [str(x)]) + metadata['user_id'].apply(lambda x: [x]) + metadata['keyword_list']

In [104]:
df = pd.read_csv("df.csv")

In [105]:
df.head()

,timestamp,uid,sid,rating,magazine_id,user_id,id,o_uid,o_sid
0,2018100100,1055,1760,1,1189,1914,@hyejinchoi_122,#b8b9d09fe2961fd62edc94912bf75a90,@hyejinchoi_122
1,2018100100,1055,8712,1,1189,1914,@hyejinchoi_112,#b8b9d09fe2961fd62edc94912bf75a90,@hyejinchoi_112
2,2018100100,1267,34504,1,5238,384,@elang8151_154,#5ddc3540a7d1d60e1dfa198787960808,@elang8151_154
3,2018100100,1267,10665,1,620,3200,@7-words_12,#5ddc3540a7d1d60e1dfa198787960808,@7-words_12
4,2018100100,1267,18639,1,2376,4236,@windyroad2_122,#5ddc3540a7d1d60e1dfa198787960808,@windyroad2_122


In [292]:
amap = dict(df[['o_sid', 'user_id']].drop_duplicates().values)

In [293]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(metadata['doc_list'])]

In [294]:
tagged_data[0]

TaggedDocument(words=['16315', '@bryceandjuli', '감정', '마음', '위로'], tags=['0'])

In [ ]:
model = Doc2Vec(tagged_data, vector_size=100, window=15, min_count=1, 
                workers=4, alpha=0.001, min_alpha=0.0001, dm=1, epochs=20, negative=10)

In [138]:
df = pd.read_csv("df.csv")
amap = dict(df[['o_sid', 'user_id']].drop_duplicates().values)
metadata['sid'] = metadata['id'].apply(lambda x: amap[x] if x in amap else None)

In [ ]:
metadata['index'] = metadata.index
metadata_p = metadata[metadata['sid'].notnull()].reset_index(drop=True)
imap = dict(metadata_p[['index', 'sid']].values)

In [220]:
from tqdm import tqdm
doc2vec_list = dict()
for doc_index in tqdm(metadata[metadata['sid'].notnull()].index): 
    new_vector = model.infer_vector(metadata['doc_list'][doc_index])
    sid = metadata['sid'][doc_index]
    
    rec_list = []
    for i, j in model.docvecs.most_similar([new_vector], topn=1000): 
        if int(i) in imap: 
            rec_list += [(int(imap[int(i)]), j)]
    rec_list = rec_list[0:100]
    rec_dict = dict(rec_list)
    doc2vec_list[int(sid)] = rec_dict

100%|██████████| 34612/34612 [12:38<00:00, 45.61it/s]


In [221]:
import pickle
with open('Tag2Vec.pkl', 'wb') as write_file:
    pickle.dump(doc2vec_list, write_file)

In [222]:
class OfflineItemSimilarity:
    def __init__(self, data_file=None, similarity_path=None, model_name='ItemCF', \
        dataset_name='Sports_and_Outdoors'):
        self.similarity_path = similarity_path
        self.model_name = model_name
        self.similarity_model = self.load_similarity_model(self.similarity_path)
        self.max_score, self.min_score = self.get_maximum_minimum_sim_scores()
        
    def get_maximum_minimum_sim_scores(self):
        max_score, min_score = -1, 100
        for item in self.similarity_model.keys():
            for neig in self.similarity_model[item]:
                sim_score = self.similarity_model[item][neig]
                max_score = max(max_score, sim_score)
                min_score = min(min_score, sim_score)
        return max_score, min_score
    
    def _convert_data_to_dict(self, data):
        """
        split the data set
        testdata is a test data set
        traindata is a train set
        """
        train_data_dict = {}
        for user,item,record in data:
            train_data_dict.setdefault(user,{})
            train_data_dict[user][item] = record
        return train_data_dict

    def _save_dict(self, dict_data, save_path = './similarity.pkl'):
        print("saving data to ", save_path)
        with open(save_path, 'wb') as write_file:
            pickle.dump(dict_data, write_file)

    def load_similarity_model(self, similarity_model_path):
        if not similarity_model_path:
            raise ValueError('invalid path')
        elif not os.path.exists(similarity_model_path):
            print("the similirity dict not exist, generating...")
            self._generate_item_similarity(save_path=self.similarity_path)
        if self.model_name in ['ItemCF', 'ItemCF_IUF', 'Item2Vec', 'LightGCN', 'Tag2Vec']:
            with open(similarity_model_path, 'rb') as read_file:
                similarity_dict = pickle.load(read_file)
            return similarity_dict
        elif self.model_name == 'Random':
            similarity_dict = self.train_item_list
            return similarity_dict

    def most_similar(self, item, top_k=1, with_score=False):
        if self.model_name in ['ItemCF', 'ItemCF_IUF', 'Item2Vec', 'LightGCN', 'Tag2Vec']:
            """TODO: handle case that item not in keys"""
            if str(item) in self.similarity_model:
                top_k_items_with_score = sorted(self.similarity_model[str(item)].items(),key=lambda x : x[1], \
                                            reverse=True)[0:top_k]
                if with_score:
                    return list(map(lambda x: (int(x[0]), (self.max_score - float(x[1]))/(self.max_score -self.min_score)), top_k_items_with_score))
                return list(map(lambda x: int(x[0]), top_k_items_with_score))
            elif int(item) in self.similarity_model:
                top_k_items_with_score = sorted(self.similarity_model[int(item)].items(),key=lambda x : x[1], \
                                            reverse=True)[0:top_k]
                if with_score:
                    return list(map(lambda x: (int(x[0]), (self.max_score - float(x[1]))/(self.max_score -self.min_score)), top_k_items_with_score))
                return list(map(lambda x: int(x[0]), top_k_items_with_score))
            else:
                item_list = list(self.similarity_model.keys())
                random_items = random.sample(item_list, k=top_k)
                if with_score:
                    return list(map(lambda x: (int(x), 0.0), random_items))
                return list(map(lambda x: int(x), random_items))
        elif self.model_name == 'Random':
            random_items = random.sample(self.similarity_model, k = top_k)
            if with_score:
                return list(map(lambda x: (int(x), 0.0), random_items))
            return list(map(lambda x: int(x), random_items))

In [223]:
similaritymodel = OfflineItemSimilarity(similarity_path='Tag2Vec.pkl', model_name='Tag2Vec')

In [224]:
import random
similaritymodel.most_similar(0, top_k=10)

[3893, 1056, 353, 296, 1067, 84, 3912, 3742, 2931, 3066]